#**Predict Mask**

1.Load Model
2. Load test dataset - Images
3. Predict the mask
4. Calculate FPS

In [1]:
# import h5py

# file_path = "/content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug/unet-non-aug.h5" # h5 파일 경로를 여기에 입력하세요.

# try:
#     with h5py.File(file_path, 'r') as file:
#         print(list(file.keys()))  # 파일 내 데이터셋 목록을 출력합니다.
#         print("H5 파일이 정상적으로 열렸습니다.")
# except Exception as e:
#     print("H5 파일을 여는데 실패했습니다. 파일이 손상되었거나 없을 수 있습니다.")
#     print("Error message:", e)

['model_weights', 'optimizer_weights']
H5 파일이 정상적으로 열렸습니다.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import**

In [20]:
import os
import time
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf

**Class Color Change**

In [21]:
# def convert_to_color_image(mask):
#     # 객체의 색상을 정의합니다. 이 예시에서는 녹색을 사용합니다.
#     object_color = (0, 255, 0)

#     # 컬러 마스크 이미지를 초기화합니다.
#     color_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)

#     # 마스크가 2차원이라면, 채널 차원을 추가합니다.
#     if len(mask.shape) == 2:
#         mask = np.expand_dims(mask, axis=-1)

#     # 객체에 색상을 적용합니다.
#     color_mask[mask == 1] = object_color

#     return color_mask

# def create_dir(path):
#     if not os.path.exists(path):
#         os.makedirs(path)

**Seeding**

In [22]:
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

**Hyperparameters**

In [23]:
height = 3072
width = 1996

**Path**

In [24]:
dataset_path = r"/content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug/new/images"
save_path = r"/content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug/new/result2"
model_path = r"/content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug/unet-non-aug.h5"

**Create folder**

In [25]:
def create_dir (path):
  if not os.path.exists(path):
    os.makedirs(path)

**Folder to save predicted mask**

In [26]:
create_dir(save_path)

**Load model**

In [27]:
model = tf.keras.models.load_model(model_path)

In [28]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 384, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 384, 256, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 384, 256, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 384, 256, 64)         0         ['batch_normalization[0][0]

**Load test Dataset: Images**

In [29]:
test_x = sorted(glob(os.path.join(dataset_path, "*.jpg")))
print(f"Test Images: {len(test_x)}")

Test Images: 16


In [30]:
test_x[0]

'/content/drive/MyDrive/006_BMS2/0002_latxray_training/dataset/non-aug/new/images/f00165953-221010.jpg'

**Predict the mask & Calculate Fps**

In [35]:
time_taken = []

for x in tqdm(test_x):
  name = x.split("/")[-1]

  x = cv2.imread(x, cv2.IMREAD_COLOR)
  x = cv2.resize(x, (256, 384)) # 이미지 크기 조정
  x = x/255.0
  x = np.expand_dims(x, axis = 0)

  start_time = time.time()
  p = model.predict(x)[0]
  total_time = time.time() - start_time
  time_taken.append(total_time)

  p = p > 0.5
  p = p * 255
  p = p.astype(np.uint8)  # 정수형으로 변환

  cv2.imwrite(os.path.join(save_path, name), p)

  0%|          | 0/16 [00:00<?, ?it/s]

1/1 [==============================] - 0s 20ms/step


  0%|          | 0/16 [00:00<?, ?it/s]


error: ignored

In [ ]:
mean_time = np.mean(time_taken)
mean_fps = 1/mean_time

In [ ]:
# print(f"Mean Time: {mean_time: 1.5f} - Mean FPS: {mean_fps: 2.5f}")
print(f"Mean Time: {mean_time:.5f} - Mean FPS: {mean_fps:.5f}")